In [1]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = '/content/drive/My Drive/'

Mounted at /content/drive


In [4]:
import zipfile

def extract_zip_file(filename):
    
    if not zipfile.is_zipfile(filename):
        raise ValueError('Your file has wrong extension.')
    
    z = zipfile.ZipFile(filename, 'r')
    z.extractall()


test_opus_filename = '/content/drive/My Drive/test_opus.zip'
extract_zip_file(test_opus_filename)

train_opus_filename = '/content/drive/My Drive/train_opus.zip'
extract_zip_file(train_opus_filename)

In [ ]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

## Install NeMo
!python -m pip install nemo_toolkit[all]

In [46]:
import glob
import os
import subprocess
import tarfile
import wget
from tqdm.notebook import tqdm


# Папка, где будет размещаться датасет Golos
data_dir = '/content'

def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
  else:
      golos_path = os.path.join(data_dir, fname)
  return golos_path

In [58]:
import nemo
import nemo.collections.asr as nemo_asr

import logging
logging.getLogger('nemo_logger').setLevel(logging.ERROR)

In [9]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from("QuartzNet15x5_golos.nemo")

[NeMo W 2021-11-30 11:43:57 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-30 11:43:57 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-30 11:43:57 features:265] PADDING: 16
[NeMo I 2021-11-30 11:43:57 features:282] STFT using torch
[NeMo I 2021-11-30 11:43:58 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


In [10]:
import torch

# load state dict from Pytorch checkpoint (.ckpt)

state_dict = torch.load('drive/MyDrive/epoch=0-step=1547.ckpt')['state_dict']
preprocessor = {v.split('preprocessor.')[-1] : k for (v, k) in state_dict.items() if v.startswith('preprocessor')}
asr_model.preprocessor.load_state_dict(preprocessor)

encoder = {v[8:] : k for (v, k) in state_dict.items() if v.startswith('encoder')}
asr_model.encoder.load_state_dict(encoder)

decoder = {v.split('decoder.')[-1] : k for (v, k) in state_dict.items() if v.startswith('decoder')}
asr_model.decoder.load_state_dict(decoder)

<All keys matched successfully>

In [53]:
import json


# load test manifest
manifest_path = os.path.split(os.path.abspath("test_opus/farfield/manifest.jsonl"))[0]
with open("test_opus/farfield/manifest.jsonl", 'r') as fin:
    with open("test_opus/farfield/manifest_abs.jsonl", 'w') as fout:
        for line in fin:
            metadata = json.loads(line)
            metadata["audio_filepath"] = os.path.join(manifest_path, metadata["audio_filepath"])
            json.dump(metadata, fout)
            fout.write('\n')


# load val manifest
i = 0
manifest_path = os.path.split(os.path.abspath("test_opus/farfield/manifest.jsonl"))[0]
with open("test_opus/farfield/manifest.jsonl", 'r') as fin:
    with open("test_opus/farfield/manifest_val_abs.jsonl", 'w') as fout:
        for line in fin:
            if i < 10:
                i += 1
                metadata = json.loads(line)
                metadata["audio_filepath"] = os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
            else:
                break


# load train manifest
i = 0
manifest_path = os.path.split(os.path.abspath("/content/drive/My Drive/manifest.jsonl"))[0]
with open("/content/drive/My Drive/manifest.jsonl", 'r') as fin:
    with open("/content/drive/My Drive/manifest_abs.jsonl", 'w') as fout:
        for line in fin:
            metadata = json.loads(line)
            if metadata["audio_filepath"].startswith('crowd'):
                continue
            
            if i < 62000:  # just left 50% data
                i += 1
                metadata["audio_filepath"] = os.path.join("/content/", metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
            else:
                break

In [13]:
#! head "/content/drive/My Drive/manifest_abs.jsonl" # train

In [15]:
#! head "test_opus/farfield/manifest_abs.jsonl" # test

In [54]:
train_manifest = "/content/drive/My Drive/manifest_abs.jsonl"
test_manifest = "test_opus/farfield/manifest_abs.jsonl"
val_manifest = "test_opus/farfield/manifest_val_abs.jsonl"

In [29]:
def load_jsonl(path):
    data = {}
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            dict_manifest = json.loads(line)
            file = dict_manifest["id"] + ".opus"
            data[file] = dict_manifest["text"]
    return data

len(load_jsonl(train_manifest))

62000

In [49]:
#train_path = '/content/farfield/'
#filenames = os.listdir(train_path)
#train_files = [train_path + filename for filename in filenames]
#
#pairs = []
#
#for fname in tqdm(train_files):
#    prediction = asr_model.transcribe(paths2audio_files=[fname], batch_size=1)[0]
#    filename = fname.split('/content/farfield/')[-1]
#    real_transcription = ground_truth_train[filename]
#    pairs.append((fname, real_transcription, prediction))
#
#print('*****')

In [55]:
params_model = asr_model._cfg

params_model['train_ds']['parser'] = 'base'
params_model['validation_ds']['parser'] = 'base'
params_model['test_ds']['parser'] = 'base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size'] = 16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size'] = 16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size'] = 16

params_model['optim']['lr'] = 0.0005

In [56]:
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint


class CusModelCheckpoint(ModelCheckpoint):
    
    def __init__(self):
        pass

    def on_validation_end(self, trainer, pl_module):
        self.save_checkpoint(trainer, pl_module)

    def on_train_epoch_end(self, trainer, pl_module, outputs):
        self.save_checkpoint(trainer, pl_module)
    
ckpt_callback = CusModelCheckpoint()

In [57]:
asr_model.cuda()

asr_model.setup_optimization(optim_config=DictConfig(params_model['optim']))
asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

trainer = pl.Trainer(gpus=1, max_epochs=1, val_check_interval=0.1)
trainer.fit(asr_model)

[NeMo W 2021-11-30 12:14:30 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 12:14:30 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 12:14:30 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-11-30 12:14:39 collections:173] Dataset loaded with 61998 files totalling 72.20 hours
[NeMo I 2021-11-30 12:14:39 collections:174] 2 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 12:14:39 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


...

[NeMo W 2021-11-30 12:14:39 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:16:26 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


Training: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:17:08 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


If you'd like to save this model checkpoint for loading later (e.g. for fine-tuning, or for continuing training), you can simply call `first_asr_model.save_to(<checkpoint_path>)`. Then, to restore your weights, you can rebuild the model using the config (let's say you call it `first_asr_model_continued` this time) and call `first_asr_model_continued.restore_from(<checkpoint_path>)`.


Below is an example of a simple inference loop in pure PyTorch. It also shows how one can compute Word Error Rate (WER) metric between predictions and references.

In [61]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
asr_model.setup_test_data(test_data_config=params_model['test_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.5328017159760136
